# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109B Data Science 2: Advanced Topics in Data Science 

##  Final Project: Milestone 3 - Final Project [70 pts]


**Harvard University**<br/>
**Spring 2020**<br/>
**Group Members**: Fernando Medeiros, Mohammed Gufran Pathan, and Prerna Aggarwal<br/>

<hr style="height:2pt">

---

In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML, display
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

<div class="theme"> Final Deliverables </div>

1. **Code Report:** You are expected to submit the code you developed as part of the course project. The commented code should be provided in report format. This means that each group in a Jupyter notebook should explain—in a clean and concise report fashion—how they proceeded at every step and coding/methodology  choices . The code report should have a structure that consists of an introduction, body and conclusion.
1. **Ignite Talk:** You will present the talk on 5/11, 5/12, or 5/13. Details to come for Ignite Talk guidelines.

[Final Project Guidelines](https://docs.google.com/document/d/1Zhmm9JP4FGQBi5abFiM22e5iXYo_rr7i_vbpW0-xt8A/edit)

In [2]:
# Baisc Python Libraries
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

plt.style.use("ggplot")

In [3]:
# Project Python Lybraries
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from pandas.io.json import json_normalize
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import json
import lxml
import lzma
import nltk
import tensorflow as tf
from tensorflow.keras import backend
from tensorflow.keras.layers import Input, SimpleRNN, Embedding, Dense, TimeDistributed, GRU,\
                                    Dropout, Bidirectional, Conv1D, BatchNormalization
from tensorflow.keras.models import Model, Sequential,model_from_json
from tensorflow.keras.utils import to_categorical

print(tf.keras.__version__)
print(tf.__version__)

2.2.4-tf
2.0.0


In [4]:
# Set seed for repeatable results
np.random.seed(123)
tf.random.set_seed(456)

<div class="theme"> Part 1: Data Prep </div>
Read `north_carolina.xz` into a pandas dataframe using the provided code below.

In [5]:
base_path = "./data/text"
state = 'north_carolina_text.xz'
f = lzma.open(os.path.join(base_path, state), "rb")
state_data = f.readlines()
f.close()

In [6]:
data_json = [json.loads(line) for line in state_data]
print(f'Flattening data for {state}')
data = json_normalize(data_json)

Flattening data for north_carolina_text.xz


In [7]:
# fix date
data['decision_date_p'] = pd.to_datetime(data.decision_date, errors='coerce')
# add year of decision date to dataset
data['decision_year'] = data.decision_date_p.dt.year
# add month of decision date to dataset
data['decision_month'] = data.decision_date_p.dt.month

In [8]:
# select one year of cases
data_2015 = data[(data.decision_year==2015) & (data.decision_month==1)]

In [9]:
# view top 5 cases
data_2015.head()

,id,url,name,name_abbreviation,decision_date,docket_number,first_page,last_page,citations,cites_to,...,casebody.status,casebody.data.head_matter,casebody.data.opinions,casebody.data.attorneys,casebody.data.judges,casebody.data.corrections,casebody.data.parties,decision_date_p,decision_year,decision_month
92741,4329281,https://api.capapi.org/v1/cases/4329281/,State v. Robert McPhail,State v. McPhail,2015-01-22,341P14,811,811,"[{'type': 'official', 'cite': '367 N.C. 811'}]",[],...,ok,341P14\nState v. Robert McPhail,"[{'type': 'majority', 'text': 'Def’s PDR Under...",[],"[Ervin, J., recused]",,[State v. Robert McPhail],2015-01-22,2015.0,1.0
92758,4332768,https://api.capapi.org/v1/cases/4332768/,"Clorey Eugene France v. Glover & Petersen, P.A...","France v. Glover & Petersen, P.A.",2015-01-22,452P14,813,813,"[{'type': 'official', 'cite': '367 N.C. 813'}]",[],...,ok,452P14\nClorey Eugene France v. Glover & Peter...,"[{'type': 'majority', 'text': '1. Plaintiffs P...",[],"[Ervin, J., recused]",,"[Clorey Eugene France v. Glover & Petersen, P....",2015-01-22,2015.0,1.0
92770,4325043,https://api.capapi.org/v1/cases/4325043/,State v. Joshua Andrew Stepp,State v. Stepp,2015-01-22,038A14,809,809,"[{'type': 'official', 'cite': '367 N.C. 809'}]",[],...,ok,038A14\nState v. Joshua Andrew Stepp,"[{'type': 'majority', 'text': '1. Def’s Motion...",[],[],,[State v. Joshua Andrew Stepp],2015-01-22,2015.0,1.0
92775,4323615,https://api.capapi.org/v1/cases/4323615/,State v. Raymond Dakim Harris Joiner,State v. Joiner,2015-01-22,514P13-3,814,814,"[{'type': 'official', 'cite': '367 N.C. 814'}]",[],...,ok,514P13-3\nState v. Raymond Dakim Harris Joiner,"[{'type': 'majority', 'text': '1. Def’s Pro Se...",[],[],,[State v. Raymond Dakim Harris Joiner],2015-01-22,2015.0,1.0
92827,4326314,https://api.capapi.org/v1/cases/4326314/,State v. Reginald U. Fullard,State v. Fullard,2015-01-22,475P14,814,814,"[{'type': 'official', 'cite': '367 N.C. 814'}]",[],...,ok,475P14\nState v. Reginald U. Fullard,"[{'type': 'majority', 'text': '1. Def’s Pro Se...",[],[],,[State v. Reginald U. Fullard],2015-01-22,2015.0,1.0


In [10]:
data_2015['casebody.data.opinions'].iloc[0]

[{'type': 'majority',
  'text': 'Def’s PDR Under N.C.G.S. § 7A-31 (COA13-1182)\nDenied\nErvin, J., recused',
  'author': None}]

In [11]:
# case with id 4325043 has no author
data_2015[data_2015.id==4325043]['casebody.data.opinions'].iloc[0]

[{'type': 'majority',
  'text': '1. Def’s Motion to Strike Part B of the State’s Brief as Outside the Scope of an Appeal of Right (COA13-46)\n2. State’s Petition in the Alternative for Writ of Certiorari to Review Decisions of COA\n1. Dismissed as moot\n2. Dismissed as moot',
  'author': None}]

In [12]:
# case with id 4325043 has no author and no headnotes
data_2015[data_2015.id==4325043]['casebody.data.head_matter'].iloc[0]

'038A14\nState v. Joshua Andrew Stepp'

In [13]:
# case with id 4325043 has no author and no judges
data_2015[data_2015.id==4325043]['casebody.data.judges'].iloc[0]

[]

In [14]:
data_2015[data_2015.id==4317084]['casebody.data.opinions'].iloc[0]

[{'type': 'majority',
  'text': 'MARTIN, Chief Justice.\nDefendant grew marijuana in view of his driveway, leaving three potted plants exposed to any visitor who might approach his residence. Two detectives did just that, and when they saw the plants, they seized them before returning the following day with a warrant to search defendant’s home. At trial, the court denied defendant’s motion to suppress the evidence of the seized plants. On appeal, the Court of Appeals reversed the trial court. We now reverse the decision of the Court of Appeals.\nOn 5 May 2011, the Johnston County Sheriff’s Office received an anonymous tip that Jerry Grice, Jr. was growing marijuana at a particular residence on Old School Road. In response, the Sheriff’s Office dispatched two detectives, Guseman and Allen, to conduct a knock and. talk investigation at the address. Both detectives had extensive training in narcotic investigations, including training in identifying marijuana. The property was located in a

In [15]:
data_2015[data_2015.id==4317084]['casebody.data.head_matter'].iloc[0]

'STATE OF NORTH CAROLINA v. JERRY WADE GRICE, JR.\nNo. 501PA12\n(Filed 23 January 2015)\n1. Search and Seizure — motion to suppress — plain view doctrine — officers’ access to contraband\nIt was not error for the trial court to deny defendant’s motion to suppress evidence of marijuana plants seized without a warrant from an area outside of his home in view of his driveway. The investigating officers had a lawful right of access from the driveway to the marijuana plants, which were approximately fifteen yards away in an unfenced area bordering a wood line. Even assuming the plants were in the home’s curtilage, the officers did not violate the Fourth Amendment by traveling from one portion of the curtilage to another to seize the plants that were in plain view. The seizure was a minimal intrusion on defendant’s property rights when balanced against the State’s interest in seizing contraband.\n2. Search and Seizure — motion to suppress — exigent circumstances\nIt was not error for the tri

In [16]:
# filter data and keep those cases with authors
data_f = data[data['casebody.data.judges'].notna()]

In [17]:
for i in range(6):
    data_f[f'casebody.data.opinions.text{i}'] = data_f['casebody.data.opinions'].map(lambda x: x[i]['text']\
                                                                                     if len(x)>=i+1 else None)

In [18]:
print('1:', data_f[data_f['casebody.data.opinions.text0'].notna()].iloc[0][0])
print('2:', data_f[data_f['casebody.data.opinions.text1'].notna()].iloc[0][0])
print('3:', data_f[data_f['casebody.data.opinions.text2'].notna()].iloc[0][0])
print('4:', data_f[data_f['casebody.data.opinions.text3'].notna()].iloc[0][0])
print('5:', data_f[data_f['casebody.data.opinions.text4'].notna()].iloc[0][0])
print('6:', data_f[data_f['casebody.data.opinions.text5'].notna()].iloc[0][0])
# no cases with 7 opinion
#print('7:', data_f[data_f['casebody.data.opinions.text6'].notna()].iloc[0][0])

1: 1268383
2: 11274261
3: 11272222
4: 11272060
5: 11270334
6: 11270334


In [19]:
data_f[data_f.id==4317084]['casebody.data.opinions.text0'].iloc[0]

'MARTIN, Chief Justice.\nDefendant grew marijuana in view of his driveway, leaving three potted plants exposed to any visitor who might approach his residence. Two detectives did just that, and when they saw the plants, they seized them before returning the following day with a warrant to search defendant’s home. At trial, the court denied defendant’s motion to suppress the evidence of the seized plants. On appeal, the Court of Appeals reversed the trial court. We now reverse the decision of the Court of Appeals.\nOn 5 May 2011, the Johnston County Sheriff’s Office received an anonymous tip that Jerry Grice, Jr. was growing marijuana at a particular residence on Old School Road. In response, the Sheriff’s Office dispatched two detectives, Guseman and Allen, to conduct a knock and. talk investigation at the address. Both detectives had extensive training in narcotic investigations, including training in identifying marijuana. The property was located in a rural area, and the house was s

In [20]:
print(data_f[data_f.id==11270334]['casebody.data.opinions'].str.split('.', expand=True).stack())

Series([], dtype: float64)


In [21]:
print(data_f[data_f.id==11270334]['casebody.data.opinions.text0'].str.split('.', expand=True).stack())

8310  0                                              13roWN, J
      1      \nTbe plaintiff’s evidence tends to prove that...
      2       Tbe evidence tends to prove that tbe engineer...
      3               ”\nITis Honor submitted these issues:\n1
      4       Hid defendant’s engineer, fireman or servants...
                                   ...                        
      174     But it is admitted there is no such evidence ...
      175     The entire evidence shows that it was the pur...
      176     While reasons of public policy may hold the m...
      177    \nFor the reasons given, I think there should ...
      178                                                     
Length: 179, dtype: object


In [22]:
print(data_f[data_f.id==11270334]['casebody.data.opinions.text1'].str.split('.', expand=True).stack())

8310  0                                               Qlaek, O
      1                                                      J
      2      ,\nconcurring, in part: I concur in the opinio...
      3       Juskice Brown that, when the whistle is Hoavu...
      4                                            \nIn Dunn v
                                   ...                        
      434     engineer and other employees was wanton and w...
      435     There was no evidence of contributory neglige...
      436    \nIt is true that this is not a public service...
      437     The public is entitled to use its public road...
      438                                                     
Length: 439, dtype: object


In [42]:
#import nltk.data
#nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentences = tokenizer.tokenize(str(data_f['casebody.data.opinions.text0']))
#from nltk import sent_tokenize
#sentences = sent_tokenize(str(data_f[data_f.id==11270334]['casebody.data.opinions.text0']))
sentences

['0        A majority of the court, therefore, being of t...\n1        PER CURIAM.\\nIf the sheriff or marshal seizes ...\n2        MARSHALL.',
 'Circuit Justice.\\nThe Fortuna, a ves...\n3        Walker, J.\\nThe grievance alleged by the plain...\n4        Hoke, J.\\nOn the hearing it appeared that C. G...\n                               ...                        \n97595    PER CURIAM.\\nJustice EDMUNDS took no part in t...\n97596    HUDSON, Justice.\\nHere we are asked to determi...\n97597    1.',
 'State’s Motion for Temporary Stay (COA14-41...\n97598    1.',
 'State’s Motion for Temporary Stay (COA15-15...\n97599    ORDER\\nThis matter is before this Court on def...\nName: casebody.data.opinions.text0, Length: 97600, dtype: object']

In [49]:
data_f['casebody.data.opinions.text0'].replace('\n', ' ', regex=True, inplace=True)
data_f['casebody.data.opinions.text1'].replace('\n', ' ', regex=True, inplace=True)
data_f['casebody.data.opinions.text2'].replace('\n', ' ', regex=True, inplace=True)
data_f['casebody.data.opinions.text3'].replace('\n', ' ', regex=True, inplace=True)
data_f['casebody.data.opinions.text4'].replace('\n', ' ', regex=True, inplace=True)
data_f['casebody.data.opinions.text5'].replace('\n', ' ', regex=True, inplace=True)

In [ ]:
text0_sentence = data_f['casebody.data.opinions.text0'].str.split('. ', expand=True).stack()
text1_sentence = data_f['casebody.data.opinions.text1'].str.split('. ', expand=True).stack()
text2_sentence = data_f['casebody.data.opinions.text2'].str.split('. ', expand=True).stack()
text3_sentence = data_f['casebody.data.opinions.text3'].str.split('. ', expand=True).stack()
text4_sentence = data_f['casebody.data.opinions.text4'].str.split('. ', expand=True).stack()
text5_sentence = data_f['casebody.data.opinions.text5'].str.split('. ', expand=True).stack()

text0_sentence

In [ ]:
text0_i = text0_sentence.index.get_level_values(0)
text1_i = text1_sentence.index.get_level_values(0)
text2_i = text2_sentence.index.get_level_values(0)
text3_i = text3_sentence.index.get_level_values(0)
text4_i = text4_sentence.index.get_level_values(0)
text5_i = text5_sentence.index.get_level_values(0)

text0_i

In [ ]:
data_text0_sentence = data_f.loc[text0_i].copy()
data_text1_sentence = data_f.loc[text1_i].copy()
data_text2_sentence = data_f.loc[text2_i].copy()
data_text3_sentence = data_f.loc[text3_i].copy()
data_text4_sentence = data_f.loc[text4_i].copy()
data_text5_sentence = data_f.loc[text5_i].copy()

data_text0_sentence

In [ ]:
data_text0_sentence['casebody.data.opinions.sentence0'] = text0_sentence.values
data_text1_sentence['casebody.data.opinions.sentence1'] = text1_sentence.values
data_text2_sentence['casebody.data.opinions.sentence2'] = text2_sentence.values
data_text3_sentence['casebody.data.opinions.sentence3'] = text3_sentence.values
data_text4_sentence['casebody.data.opinions.sentence4'] = text4_sentence.values
data_text5_sentence['casebody.data.opinions.sentence5'] = text5_sentence.values

data_text0_sentence['casebody.data.opinions.sentence0']

In [ ]:
data_text0_sentence.head()

In [ ]:
data_text0_sentence[data_text0_sentence.id==4317084]

In [ ]:
data_text0_sentence[data_text0_sentence.id==4317084]['casebody.data.opinions.sentence0']

In [ ]:
data[data.id==4317084]['casebody.data.opinions'].iloc[0]